# Lesson 1: Batching Commands with Pipelines

# Introduction to Batching Commands with Pipelines

Welcome! In this lesson, we are going to delve into a feature of Redis that can significantly boost your application's performance — **pipelines**. Pipelines allow you to send multiple commands to the Redis server without waiting for a response after each command. Instead, you collect a batch of commands and send them all at once, then read all the replies in a single step. This approach can make your application more efficient and responsive.

Ready to optimize your Redis usage? Let's get started!

## What You'll Learn
In this lesson, we will explore how to use Redis pipelines to batch commands. Specifically, you will learn how to:

- 🛠️ Initialize a Redis connection and pipeline.
- 📊 Batch multiple commands together.
- 🚀 Execute the batched commands efficiently.

Here's a quick example to give you an overview. Consider a scenario where you need to update the number of courses completed and set a user's name. Normally, you would execute these commands one by one. With pipelines, you can batch them like this:

```python
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Initialize values
client.set('user', '')
client.set('courses_completed', 1)

# Use the pipeline without a context manager
pipe = client.pipeline()
pipe.incr('courses_completed')
pipe.set('user', 'John')
try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction error: {e}")
finally:
    pipe.close()

# Retrieve and print updated values
courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
```

This sample code demonstrates how to connect to Redis, batch commands in a pipeline, and then execute them all together for better performance.

### Explanation:
- First, we create a pipeline and add the commands to increment the number of courses completed and set the user's name.
- Then, we execute the pipeline and print the results.
- Finally, we retrieve the updated values and display them.

Notice that we use the `pipe.execute()` method to execute the batched commands. This method sends all the commands to the Redis server and retrieves the results in a single step.

After that, we use the `pipe.close()` method to close the pipeline and release the resources. This is important to avoid memory leaks and ensure proper cleanup. Another useful method is `pipe.reset()`, which resets the pipeline and the intermediate state, but you can still reuse it.

### Transaction Results:
Let's understand the successful transaction result, which will be:

```
Transaction results: [2, True]
```

- The first element is the result of the `incr` command, which increments the value by 1, hence the value `2`.
- The second element is the result of the `set` command, which returns `True` to indicate success.

## Why It Matters

Efficiency is key in any application, and being able to execute multiple Redis commands in one go can save you a lot of time and resources. This is particularly important in real-time applications where latency can be a critical factor. By mastering pipelines, you can enhance the responsiveness of your applications and provide a smoother user experience.

Exciting, right? Ready to see how much you can optimize your Redis interactions? Let's move on to the practice section and put these concepts into action!


## Running a Redis Transaction

Great job understanding the basics of Redis transactions! Let's run the code you saw in the lesson to see it in action.

Redis transactions allow you to execute a batch of commands as a single unit. In the code, we set some initial values, add a new user, and increment the number of completed courses within a single transaction.

The commands in the pipeline are executed together without waiting for the response to each command.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

client.set('user', '')
client.set('courses_completed', 1)

pipe = client.pipeline()
pipe.set('user', 'John')
pipe.incr('courses_completed')

try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')

```

## Adding a Decrement in Pipeline

Nice progress! You’ve got the hang of executing Redis transactions within a pipeline. Now, let’s add a twist.

Rather than just setting a value and incrementing it, let's include a decrement operation in our pipeline. Modify the given code to decrement the missions_ongoing key by 2 in the pipeline.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

client.set('user', '')
client.set('courses_completed', 1)
client.set('missions_ongoing', 5)

pipe = client.pipeline()
pipe.incr('courses_completed')
pipe.set('user', 'John')
# TODO: Add a command to decrement missions_ongoing by 2 in the pipeline

try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')
missions_ongoing = client.get('missions_ongoing').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
print(f'Missions ongoing: {missions_ongoing}')

```

Here's the modified version of your code that includes a decrement operation for the `missions_ongoing` key in the pipeline. The key is decremented by 2 as requested:

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# Initialize values
client.set('user', '')
client.set('courses_completed', 1)
client.set('missions_ongoing', 5)

# Create pipeline
pipe = client.pipeline()

# Add commands to the pipeline
pipe.incr('courses_completed')       # Increment courses_completed by 1
pipe.set('user', 'John')             # Set the user's name
pipe.decrby('missions_ongoing', 2)   # Decrement missions_ongoing by 2

try:
    # Execute all the batched commands
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction error: {e}")
finally:
    pipe.close()

# Retrieve and print updated values
courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')
missions_ongoing = client.get('missions_ongoing').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
print(f'Missions ongoing: {missions_ongoing}')
```

### Key Changes:
- Added `pipe.decrby('missions_ongoing', 2)` to decrement `missions_ongoing` by 2 within the pipeline.

### Expected Transaction Results:
```
Transaction results: [2, True, 3]
```
- The first result (`2`) is from incrementing `courses_completed`.
- The second result (`True`) is from setting the user to "John".
- The third result (`3`) is from decrementing `missions_ongoing` (5 - 2 = 3).

### Final Output:
- `Courses completed: 2`
- `User: John`
- `Missions ongoing: 3`

This approach shows how to mix increment, set, and decrement operations in a single Redis pipeline!

## Debug Redis Pipeline Errors

You've made great progress so far! Now, let's debug a transactional issue in the code.

In this task, we are trying to set the user name and courses completed via a transaction. The code uses a pipeline to execute the transaction.

However, the transaction result Transaction results: [2] indicates that only the number of courses completed is incremented, and the user name is not set. Can you identify the issue and fix it?

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

client.set('user', '')
client.set('courses_completed', 1)

pipe = client.pipeline()
pipe.incr('courses_completed')
client.set('user', 'John')

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
```

The issue here arises because the code is not executing the `client.set('user', 'John')` operation within the transaction pipeline. Only the `pipe.incr('courses_completed')` operation is being executed as part of the transaction, and that’s why only the number of courses completed is incremented, but the user name is not set.

To fix this, the `set('user', 'John')` operation should also be added to the pipeline, ensuring that both operations happen within the transaction.

Here's the corrected code:

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# Initialize user and courses_completed values
client.set('user', '')
client.set('courses_completed', 1)

# Start a transaction pipeline
pipe = client.pipeline()

# Add both operations to the transaction pipeline
pipe.incr('courses_completed')
pipe.set('user', 'John')

try:
    # Execute the transaction
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

# Fetch updated values
courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

# Output the final results
print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
```

### Key Fix:
- **Added `pipe.set('user', 'John')`**: This ensures that setting the user name happens within the transaction, so it is executed together with the increment operation.

### Expected Output:
Both the user name and the number of courses completed should be updated:

```
Transaction results: [2, True]
Courses completed: 2
User: John
```

## Complete the Redis Pipeline

Wonderful progress so far! Your next task is to complete the given code. You need to batch several commands in a single pipeline.

Follow the instructions in the starter code to complete the task.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

client.set('user', '')
client.set('courses_completed', 1)

pipe = client.pipeline()

# TODO: Increment 'courses_completed' in the transaction

# TODO: Set the user name as 'John' in the transaction

try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
```

To complete the task, we need to follow the instructions and batch multiple commands into a single transaction pipeline. This involves incrementing the `courses_completed` value and setting the `user` name in the transaction, both within the pipeline.

Here’s the completed code:

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# Initializing values
client.set('user', '')
client.set('courses_completed', 1)

# Start a transaction pipeline
pipe = client.pipeline()

# TODO: Increment 'courses_completed' in the transaction
pipe.incr('courses_completed')

# TODO: Set the user name as 'John' in the transaction
pipe.set('user', 'John')

try:
    # Execute the pipeline (transaction)
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    # Closing the pipeline
    pipe.close()

# Fetch and display updated values
courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
```

### Key Changes:
1. **`pipe.incr('courses_completed')`**: This increments the number of courses completed within the transaction.
2. **`pipe.set('user', 'John')`**: This sets the user’s name to 'John' as part of the same transaction.

### Expected Output:
The output will show the results of both operations and the updated values:

```
Transaction results: [2, True]
Courses completed: 2
User: John
```

With these changes, both operations are correctly executed in a single batch transaction, ensuring atomicity.

## Write a Redis Pipeline Script

Great progress! Now, let's put everything you've learned into practice.

Write a Redis pipeline script from scratch. The script should set the initial values for product and stock, then batch commands in a pipeline to update these values together. Specifically, you should decrement stock and set product to Laptop.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# TODO: Set initial values for 'product' and 'stock' to empty string and 10, respectively.

# TODO: Create a pipeline.

# TODO: Decrement 'stock' by 1 and set 'product' to 'Laptop' in the pipeline.

# TODO: Execute the pipeline and handle exceptions. Remember to close it in `finally` block

# TODO: Retrieve and print updated values of 'stock' 

```

Here’s a complete Redis pipeline script based on your requirements. It sets the initial values for `product` and `stock`, then batches commands in a pipeline to decrement the stock and set the product to "Laptop".

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# TODO: Set initial values for 'product' and 'stock' to empty string and 10, respectively.
client.set('product', '')
client.set('stock', 10)

# TODO: Create a pipeline.
pipe = client.pipeline()

# TODO: Decrement 'stock' by 1 and set 'product' to 'Laptop' in the pipeline.
pipe.decr('stock', 1)  # Decrement stock by 1
pipe.set('product', 'Laptop')  # Set product to 'Laptop'

try:
    # TODO: Execute the pipeline and handle exceptions.
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction failed: {e}")
finally:
    # Remember to close the pipeline
    pipe.close()

# TODO: Retrieve and print updated values of 'stock' and 'product'.
stock = client.get('stock').decode('utf-8')
product = client.get('product').decode('utf-8')

print(f'Stock: {stock}')
print(f'Product: {product}')
```

### Breakdown:
1. **Setting Initial Values**: The script sets `product` to an empty string and `stock` to 10.
2. **Creating a Pipeline**: A pipeline is created to group multiple commands.
3. **Batching Commands**: The `pipe.decr('stock', 1)` command decreases the stock by 1, and `pipe.set('product', 'Laptop')` sets the product to "Laptop".
4. **Executing the Pipeline**: The `pipe.execute()` ensures that both commands are executed together atomically.
5. **Fetching Updated Values**: The updated values of `stock` and `product` are fetched and printed.

### Expected Output:
The output should show the transaction results and the updated values of `stock` and `product`.

```
Transaction results: [9, True]
Stock: 9
Product: Laptop
```

## Improve Efficiency with Redis Pipelines

You've learned about batch processing with pipelines in Redis. Now, let's make it more efficient.

Modify the code so that it uses a pipeline to set 100 keys with their respective values in a single call to the Redis server and prints the results of the pipeline execution.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

iterations = 100

for i in range(iterations):
    client.set(f'key{i}', f'value{i}')

```

To make the process more efficient using Redis pipelines, we can batch the `SET` operations into a single pipeline execution, rather than making 100 individual calls to the Redis server. Here's the modified code:

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

iterations = 100

# Create a pipeline
pipe = client.pipeline()

# Add multiple SET commands to the pipeline
for i in range(iterations):
    pipe.set(f'key{i}', f'value{i}')

try:
    # Execute all commands in a single call
    results = pipe.execute()
    print(f"Pipeline execution results: {results}")
except Exception as e:
    print(f"Pipeline execution failed: {e}")
finally:
    # Close the pipeline
    pipe.close()

# Optional: Verify one of the keys
print(f'key0: {client.get("key0").decode("utf-8")}')
```

### Breakdown of Changes:
1. **Pipeline Creation**: A pipeline is created using `client.pipeline()`.
2. **Batching Commands**: Instead of calling `client.set()` in each iteration, we add the `SET` commands to the pipeline in a loop.
3. **Pipeline Execution**: The `pipe.execute()` sends all the `SET` commands in a single call to the Redis server, which improves efficiency by reducing network overhead.
4. **Result Printing**: After the execution, the results of each command in the pipeline are printed.

### Expected Output:
The output will show the results of the pipeline execution and confirm that one of the keys has been correctly set:

```
Pipeline execution results: [True, True, ..., True]  # 100 'True' results for each SET command
key0: value0
``` 

This approach significantly reduces the number of network round-trips between your application and the Redis server, improving performance, especially for large batch operations.